In [17]:
import ctypes

from eval import *

class BtHeader(ctypes.Structure):
    _fields_ = [
        ("dwMagicCode", ctypes.c_uint32),   # header, __MGAIC_CODE__
        ("IBT_size", ctypes.c_int32),
        ("IRQ_size", ctypes.c_int32),
        ("TASK_size", ctypes.c_int32),
        ("LOOP_size", ctypes.c_int32),
        ("INS_tag_size", ctypes.c_int32),
        ("PendSV", ctypes.c_uint32),
        ("IBT_entry", ctypes.POINTER(ctypes.c_uint32)),
        ("IRQ_entry", ctypes.POINTER(ctypes.c_uint32)),
        ("TASK_entry", ctypes.POINTER(ctypes.c_uint32)),
        ("LOOP_entry", ctypes.POINTER(ctypes.c_uint32)),
        ("INS_tag", ctypes.POINTER(ctypes.c_uint8)),
    ]

In [18]:
def parse_medata_from_file(file_path):
    with open(file_path, "rb") as f:
        # Read the binary data from the file
        bin_data = f.read()

    # Create a buffer using the binary data and cast it to the BtHeader structure
    bt_header = ctypes.cast(ctypes.create_string_buffer(bin_data), ctypes.POINTER(BtHeader)).contents
    rest_bin_data = bin_data[48:]
    IBT_data_pos = bt_header.IBT_size*4
    IBT_data = rest_bin_data[:IBT_data_pos]
    other_data = rest_bin_data[IBT_data_pos:]

    return bt_header, IBT_data, other_data, bin_data

def bt_header_to_bytes(bt_header):
    return bytes(ctypes.string_at(ctypes.pointer(bt_header), 48))

In [19]:
import struct

def sort_bin(x, y):
    # Convert y to an integer
    y_int = int(y[2:].decode(), 16)

    # Convert y_int to a little-endian bytearray
    y_little_endian = bytearray(y_int.to_bytes(4, 'little'))

    # Concatenate x and y_little_endian
    if x.find(y_little_endian) == -1:
        x = x + y_little_endian

    # Convert x to a list of little-endian integers
    num_integers = len(x) // 4
    integers = struct.unpack(f'<{num_integers}I', x)

    # Sort the list of integers
    sorted_integers = sorted(integers)

    # Convert the sorted list back to a little-endian bytearray
    sorted_x = struct.pack(f'<{num_integers}I', *sorted_integers)

    print("Sorted x:", sorted_x)
    return sorted_x


In [20]:
# Replace 'your_file.bin' with the actual binary file path
name_list = ['bubblesort', 'crc32', 'dijkstra', 'edn', 'fasta', 'frac', 'levenshtein', 'nbody', 'ndes', 'nettle_aes', 'picojpeg', 'qrduino', 'rijndael', 'sglib_arraybinsearch', 'sglib_dllist', 'sglib_hashtable','sglib_listsort', 'sglib_queue', 'sglib_rbtree', 'st', 'whetstone']
# , 'Sherloc_Blinky_ns_10ms', 'Sherloc_Blinky_ns_100ms', 'FreeRTOS_MPU_ns_1', 'FreeRTOS_MPU_ns_led']

# name_list = ['bubblesort', 'crc32']
opt_level = ['O3', 'Oz']
mask = 0x0000ffff
mask2 = 0xffff0000
lib = 0x0432043c
lib2 = 0x004b

def check_IBT(file_name):
    file_path_Oz = '../../Example/out/eval/Oz/metadata/' + file_name + '.bin'
    file_path_O3 = '../../Example/out/eval/O3/metadata/' + file_name + '.bin'
    bt_header_Oz, IBT_data_Oz, other_data, bin_data = parse_medata_from_file(file_path_Oz)
    bt_header_O3, IBT_data_O3, other_data, bin_data = parse_medata_from_file(file_path_O3)
    
    # run those line of code to clean the data (only need to run once)
    for i in range(0, len(IBT_data_Oz), 4):
        value = int.from_bytes(IBT_data_Oz[i:i+4], byteorder='little') # Use 'little' for little endian
        # if the last 16 bits are all 0 or the value is the address of lib, delete it
        print(f"0x{value:08x}")        
        if (value & mask == 0) or (value == lib):
            # reduce the IBT size
            bt_header_Oz.IBT_size -= 1
            # delete the value from IBT list
            IBT_data_Oz = IBT_data_Oz[:i] + IBT_data_Oz[i+4:]
    # write the new IBT data to the binary file
    with open(file_path_Oz, 'wb') as f:
        f.write(bt_header_to_bytes(bt_header_Oz) + IBT_data_Oz + other_data)

    print(f'{file_name}_Oz: {bt_header_Oz.IBT_size}')
    
    for i in range(0, len(IBT_data_O3), 4):
        value = int.from_bytes(IBT_data_O3[i:i+4], byteorder='little') # Use 'little' for little endian
        print(f"0x{value:08x}")
        if (value & mask == 0) or (value == lib):
            bt_header_O3.IBT_size -= 1
            # delete the value from IBT list
            IBT_data_O3 = IBT_data_O3[:i] + IBT_data_O3[i+4:]
        # write the new IBT data to the binary file
    with open(file_path_O3, 'wb') as f:
        f.write(bt_header_to_bytes(bt_header_O3) + IBT_data_O3 + other_data)
        
    print(f'{file_name}_O3: {bt_header_O3.IBT_size}')

    with open('../../Example/out/eval/ibt_size.log', 'a') as f:
        # {file_name} O3/Oz: 
        f.write(f'{bt_header_O3.IBT_size}/{bt_header_Oz.IBT_size} & ')

def print_IBT(file_name):
    file_path_Oz = '../../Example/out/eval/Oz/metadata/' + file_name + '.bin'
    file_path_Oz_new = '../../Example/out/eval/Oz/metadata_new/' + file_name + '.bin'
    file_path_O3 = '../../Example/out/eval/O3/metadata/' + file_name + '.bin'
    file_path_O3_new = '../../Example/out/eval/O3/metadata_new/' + file_name + '.bin'

    bt_header_Oz, IBT_data_Oz, other_data, bin_data = parse_medata_from_file(file_path_Oz)
    bt_header_Oz_new, IBT_data_Oz_new, other_data_new, bin_data_new = parse_medata_from_file(file_path_Oz_new)

    bt_header_O3, IBT_data_O3, other_data, bin_data = parse_medata_from_file(file_path_O3)
    bt_header_O3_new, IBT_data_O3_new, other_data_new, bin_data_new = parse_medata_from_file(file_path_O3_new)


    # Now you can access the fields of the BtHeader structure
    entry_oz = []
    entry_o3 = []
    with open('../../Example/out/eval/ibt_entry.log', 'a') as f:
        for i in range(0, len(IBT_data_Oz), 4):
            value = int.from_bytes(IBT_data_Oz[i:i+4], byteorder='little') # Use 'little' for little endian
            # if the last 16 bits are all 0 or the value is the address of lib, delete it
            entry_oz.append(value)
            f.write(f"0x{value:08x}, ")    
        f.write(f'{file_name}_Oz: {bt_header_Oz.IBT_size}\n')

        # f.write('\n') 
        for i in range(0, len(IBT_data_Oz_new), 4):
            value = int.from_bytes(IBT_data_Oz_new[i:i+4], byteorder='little') # Use 'little' for little endian
            # if the last 16 bits are all 0 or the value is the address of lib, delete it
            if value >> 16 == 0x0432:
                bt_header_Oz_new.IBT_size -= 1
                continue
            if value in entry_oz:
                bt_header_Oz_new.IBT_size -= 1
                continue
            if value:
                f.write(f"0x{value:08x}, ")
                MetaDataRefresh("../../Example/out/eval/Oz/eval_log/eval.log", file_path_Oz, IBT_data_Oz_new[i:i+4]).refresh_run()    
        f.write(f'{file_name}_Oz_new: {bt_header_Oz_new.IBT_size}\n')

        f.write('\n')
        
        for i in range(0, len(IBT_data_O3), 4):
            value = int.from_bytes(IBT_data_O3[i:i+4], byteorder='little') # Use 'little' for little endian
            entry_o3.append(value)
            f.write(f"0x{value:08x}, ")
        f.write(f'{file_name}_O3: {bt_header_O3.IBT_size}\n')
        
        for i in range(0, len(IBT_data_O3_new), 4):
            value = int.from_bytes(IBT_data_O3_new[i:i+4], byteorder='little') # Use 'little' for little endian
            if value >> 16 == 0x0432:
                bt_header_O3_new.IBT_size -= 1
                continue
            if value in entry_o3:
                bt_header_O3_new.IBT_size -= 1
                continue
            if value:
                f.write(f"0x{value:08x}, ")
                MetaDataRefresh("../../Example/out/eval/O3/eval_log/eval.log", file_path_O3, IBT_data_O3_new[i:i+4]).refresh_run()    
        f.write(f'{file_name}_O3_new: {bt_header_O3_new.IBT_size}\n')        
        
        f.write('\n')

        with open('../../Example/out/eval/ibt_size.log', 'a') as f:
        # {file_name} O3/Oz: 
            f.write(f'{bt_header_O3.IBT_size+bt_header_O3_new.IBT_size}/{bt_header_Oz.IBT_size+bt_header_Oz_new.IBT_size} & ')

def print_IBT_updated(file_name):
    file_path_Oz = '../../Example/out/eval/Oz/metadata/' + file_name + '.bin'
    file_path_O3 = '../../Example/out/eval/O3/metadata/' + file_name + '.bin'

    bt_header_Oz, IBT_data_Oz, other_data, bin_data = parse_medata_from_file(file_path_Oz)

    bt_header_O3, IBT_data_O3, other_data, bin_data = parse_medata_from_file(file_path_O3)

    # Now you can access the fields of the BtHeader structure
    entry_oz = []
    entry_o3 = []
    with open('../../Example/out/eval/ibt_entry.log', 'a') as f:
        for i in range(0, len(IBT_data_Oz), 4):
            value = int.from_bytes(IBT_data_Oz[i:i+4], byteorder='little') # Use 'little' for little endian
            # if the last 16 bits are all 0 or the value is the address of lib, delete it
            entry_oz.append(value)
            f.write(f"0x{value:08x}, ")    
        f.write(f'{file_name}_Oz: {bt_header_Oz.IBT_size}\n')

        f.write('\n')
        
        for i in range(0, len(IBT_data_O3), 4):
            value = int.from_bytes(IBT_data_O3[i:i+4], byteorder='little') # Use 'little' for little endian
            entry_o3.append(value)
            f.write(f"0x{value:08x}, ")
        f.write(f'{file_name}_O3: {bt_header_O3.IBT_size}\n')
            
        f.write('\n')

        with open('../../Example/out/eval/ibt_size.log', 'a') as f:
        # {file_name} O3/Oz: 
            f.write(f'{bt_header_O3.IBT_size}/{bt_header_Oz.IBT_size} & ')

for file_name in name_list:
    # check_IBT(file_name)
    # merge IBT entries
    # print_IBT(file_name)
    print_IBT_updated(file_name)

